# Process Notebooks

In [1]:
import string
from processors.import_processor import get_imports
import logging
import nbformat
import json
import pandas as pd
import numpy as np
import pandas as pd
import re

logging.basicConfig(filename="log/processNotebooks.log", filemode='w')
# Creating an object
logger = logging.getLogger()
# Setting the threshold of logger to DEBUG

logger.setLevel(logging.DEBUG)
text = ''

In [2]:
DATA_DIRECTORY = 'data-10k/'
BASE64_IMG_DIRECTORY = 'base64Images/'
SAMPLE_DATA = 'sample-10000.txt'

 # Lambdas and Helper methods

In [3]:
def getLanguage(x):
    """
    Takes each notebook and extracts the language information from the notebook.
    """
    language = None
    # x = row.raw
    if 'kernelspec' in x['metadata'].keys() :
        language = x['metadata']['kernelspec']['name']
    elif 'language_info' in x['metadata'].keys():
        language = x['metadata']['language_info']['name']
    else:
        logger.exception("kernelspec and language_info keys not present in metadata. printing metadata entry for notebook")
        logger.error(x['metadata'])
        language = None
    return language  

In [4]:
def getSourceCodeAndExtractImports(data_row):
    filename = data_row['fileNames']
    data = data_row['raw']
    cells = data['cells']
    language = data_row['language']
    import_strings = []
    print(f'Processing {filename}')
    if language is not None and 'py' in language:
        for cell in cells:
            if cell['cell_type'] == 'code':
                src = cell['source']
                try:
                    import_list = get_imports(src)
                    for import_item in import_list:
                        modules = import_item.module
                        imported_names = import_item.name
                        import_str = ".".join(modules) + " " + ".".join(imported_names)
                        import_strings.append(import_str)
                except:
                    pass

    return json.dumps(import_strings)

In [5]:
def hasMatplotlib(line: "str"):
    if line.strip().startswith("#"):
        return False

    # fr matplotlib
    if '.subplots(' in line:
        return True
    
    return False


def hasBokeh(line: "str"):
    if line.strip().startswith("#"):
        return False

    # for bokeh
    if 'gridplot' in line:
        return True
    if 'column(' in line:
        return True
    if 'row(' in line:
        return True
    if 'layout(' in line:
        return True
    
    return False


def hasPlotly(line: "str"):
    if line.strip().startswith("#"):
        return False
    
    # for plotly
    if 'facet_' in line: 
        return True
    
    return False


def count_factory(check):
    def fn(row):
        filename = row['fileNames']
        data = row['raw']
        cells = data['cells']
        count = 0
        for cell in cells:
            if cell['cell_type'] == 'code':
                src = cell['source']
                try:
                    for line in src.split('\n'):
                        if check(line):
                            count += 1
                except:
                    pass
            
        return count
    return fn

In [6]:
# get base64 image metadata from each cell  
def storeBase64ImagesAndExtractImageMetadata(row):
    x = row['raw']
    cells = x['cells']
    base64Images = []
    image_metadata = []
    for cell in cells:
        if cell['cell_type'] == 'code':
            # check if cell contains display type as images or plots
            if 'outputs' in cell.keys():
                outputs = cell['outputs']
                for output in outputs:
                    if 'data' in output.keys():
                        data = output['data']
                        if 'image/png' in data.keys():
                            base64Images.append(data['image/png'])
                            image_metadata.append('png')
                        elif 'image/jpeg' in data.keys():
                            base64Images.append(data['image/jpeg'])
                            image_metadata.append('jpeg')
                        elif 'image/svg+xml' in data.keys():
                            base64Images.append(data['image/svg+xml'])
                            image_metadata.append('svg')
                            
    # Store the images into a directory
    for i, image in enumerate(base64Images):
        image_file_name = f"{row['fileNames']}-{i}.png"
        try:
            with open(f"{DATA_DIRECTORY}{BASE64_IMG_DIRECTORY}{image_file_name}", "w") as f:
                f.write(image)
        except Exception as e:
            continue
    
    return json.dumps(image_metadata)

In [7]:
def fileToNbNode(x):
    a = None
    try:
        # print("loading file: data-1k/" + x)
        a = nbformat.read(f"{DATA_DIRECTORY}{x}", as_version=4)
    except Exception as e:
        # print(x)
        logger.exception("Exception occurred for file '"+ x + "': "+ repr(e))
        
        a = None
    return a

 ## load the text file into a pandas dataframe

In [8]:
fnames = []
with open(f"{SAMPLE_DATA}") as f:
    for line in f:
        fnames.append(line.strip())

# load fnames into a pandas dataframe with column "fileNames"
df = pd.DataFrame(fnames, columns=["fileNames"])
validFiles = df.dropna()
# print("number of valid files")

# for each fileName in df, load it using nbformat and store it in a new column "raw"
df['raw'] = df['fileNames'].apply(lambda x: fileToNbNode(x))
validFiles = df.dropna()
print("number of valid files")
print(validFiles.count())
# validFiles.set_option('display.width', 999)
pd.options.display.max_colwidth = 999
# print(validFiles.count())

/home/ubuntu/venv/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


number of valid files
fileNames    9949
raw          9949
dtype: int64


## Now get the kernel type and language for each file and store it in the "language" column of this dataframe.

In [9]:
validFiles['language'] = validFiles['raw'].apply(lambda x: getLanguage(x) )
print(f'Shape of validfiles with language: {validFiles.shape}')
# logger.info(validFiles[['fileNames','language']].head(20))
print(f'Validfiles with drop na after language : {validFiles.shape}')
validFiles['language'].value_counts()

Shape of validfiles with language: (9949, 3)
Validfiles with drop na after language : (9949, 3)


/tmp/ipykernel_1618263/3665601246.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validFiles['language'] = validFiles['raw'].apply(lambda x: getLanguage(x) )


python3                       7662
python2                       1054
conda-root-py                  101
ir                              81
Python [Root]                   73
                              ... 
conda-env-ipykernel_py2-py       1
stat                             1
conda_nlp                        1
conda-env-ENTER-py               1
fire                             1
Name: language, Length: 386, dtype: int64

 ## Get the subplot information

In [10]:

# validFiles['subplot_count'] = validFiles.apply(lambda nb: getSourceCodeAndExtractSubplotCount(nb), axis=1)
validFiles['matplotlib_count'] = validFiles.apply(lambda nb: count_factory(hasMatplotlib)(nb), axis=1)
validFiles['plotly_count'] = validFiles.apply(lambda nb: count_factory(hasPlotly)(nb), axis=1)
validFiles['bokeh_count'] = validFiles.apply(lambda nb: count_factory(hasBokeh)(nb), axis=1)
print(validFiles.head(50))

/tmp/ipykernel_1618263/2040402051.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validFiles['matplotlib_count'] = validFiles.apply(lambda nb: count_factory(hasMatplotlib)(nb), axis=1)
/tmp/ipykernel_1618263/2040402051.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validFiles['plotly_count'] = validFiles.apply(lambda nb: count_factory(hasPlotly)(nb), axis=1)
/tmp/ipykernel_1618263/2040402051.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

                                         fileNames  \
0   b7cfce55c45a79fa534aa23dc5d8854222c8efc9.ipynb   
1   c36058450b64e13ecd3e2311b8fae51747c80581.ipynb   
2   9ea31ba5f545bdb1e0023b0289ff7308988f906d.ipynb   
3   eb40b80b135a0fec53f8e6e2eb9959b0bcbf50a4.ipynb   
4   5192b46e6a1bab87e9f4e97c06c634c2512d0f80.ipynb   
5   d0553fce82ac1f86ca64d4e03268a5483c31a2bd.ipynb   
6   1f5bdc21aad4f24bdc9f5444b4bd1b62e75df407.ipynb   
7   6ed9962478f4f7b22de249d600b414cb63f28c05.ipynb   
8   aff9a0532fb2f9254f2c391562a00b569b717ff1.ipynb   
9   5702d7f7c1138d5c67d1d13cafcb91e8073120af.ipynb   
10  0398c968c730e1a002154c37fca595391ede99db.ipynb   
11  e5d9dedec7f6a1a9469b0d6a88eb30460db5d0a3.ipynb   
12  a47f6ab35d0f3e18c903f21a74b1597597c2ecdc.ipynb   
13  de3398b8d68e74aecfcf74efb0261c08d94a032d.ipynb   
14  d2757af344727f6d2f5560aa69faefcd573cd5bf.ipynb   
15  20f7ced5aed993f213bda8d718124f7c6efa6262.ipynb   
16  ae6f1f8330f1b932fc9b66bf15177dedf68d481b.ipynb   
17  dc669808f556af85dddfd521

In [ ]:
validFiles['imports'] = validFiles.apply(lambda nb: getSourceCodeAndExtractImports(nb), axis=1)
# languageGroups = validFiles.groupby('language').count()
# logger.info(languageGroups.head(10))
logger.info(" number of files after dropping values with no language_info is")
logger.info(validFiles.count())

# validFiles['source'] = validFiles['raw'].apply(lambda x: getSourceFromCells(x))
# validFiles['output'] = validFiles['raw'].apply(lambda x: getOutputFromCells(x))
# validFiles= validFiles.dropna()
# logger.info("valid files with code cells with outputs")
# logger.info(validFiles.count())
validFiles['images'] = validFiles.apply(lambda row: storeBase64ImagesAndExtractImageMetadata(row), axis=1)
validFiles = validFiles.dropna()
logger.info("valid files with base64 image outputs")
logger.info(validFiles.count())
validFiles['numImages'] = validFiles['images'].apply(lambda x: len(x))
logger.info("sum of numImages column")
logger.info(validFiles['numImages'].sum())

logger.info(validFiles[['fileNames','language','numImages']].head(5))

Processing b7cfce55c45a79fa534aa23dc5d8854222c8efc9.ipynb
Processing c36058450b64e13ecd3e2311b8fae51747c80581.ipynb
Processing 9ea31ba5f545bdb1e0023b0289ff7308988f906d.ipynb
Processing eb40b80b135a0fec53f8e6e2eb9959b0bcbf50a4.ipynb
Processing 5192b46e6a1bab87e9f4e97c06c634c2512d0f80.ipynb
Processing d0553fce82ac1f86ca64d4e03268a5483c31a2bd.ipynb
Processing 1f5bdc21aad4f24bdc9f5444b4bd1b62e75df407.ipynb
Processing 6ed9962478f4f7b22de249d600b414cb63f28c05.ipynb
Processing aff9a0532fb2f9254f2c391562a00b569b717ff1.ipynb
Processing 5702d7f7c1138d5c67d1d13cafcb91e8073120af.ipynb
Processing 0398c968c730e1a002154c37fca595391ede99db.ipynb
Processing e5d9dedec7f6a1a9469b0d6a88eb30460db5d0a3.ipynb
Processing a47f6ab35d0f3e18c903f21a74b1597597c2ecdc.ipynb
Processing de3398b8d68e74aecfcf74efb0261c08d94a032d.ipynb
Processing d2757af344727f6d2f5560aa69faefcd573cd5bf.ipynb
Processing 20f7ced5aed993f213bda8d718124f7c6efa6262.ipynb
Processing ae6f1f8330f1b932fc9b66bf15177dedf68d481b.ipynb
Processing dc6

Processing b3fdcc89514b35439a701e529252d812464450d9.ipynb
Processing e7e4bf3e887afeaf60ef55764e2f5cb944edc110.ipynb
Processing 7cb9c5380f35a4fe0fa829587c3be9dfe1892c49.ipynb
Processing d4ca89cb8d2634545a6c8c6ae6843ad4c3b4aee5.ipynb
Processing eb645051e5783189486c4e46c643595dbf82650c.ipynb
Processing 5cbfabb2b61c9e8aa0cddf28ecb95f00441aac41.ipynb
Processing 91a48586abd4203f500151948daf48a6092532ac.ipynb
Processing d1fb9e68ec3efd38846601502741666aa8aaab0a.ipynb
Processing 2c68cb8c0e27c3695282cdd195dea14f2f0e36a8.ipynb
Processing 03b84f49b55562c0ce6cf95a8a0e91b25bf184b6.ipynb
Processing 95e1038109f0a7df6cb09e25ba9c44121f4bc4dc.ipynb
Processing 739b3fb6ded11eb4cfa07def92fc243f023a955d.ipynb
Processing 1851894fca13ab8c56dc52f0b4c639eae40a97e5.ipynb
Processing e98cc1207d6f942578313f8fa4c9038325b924b8.ipynb
Processing be3649dd7509b32852284b3f9321204ff6e4bebd.ipynb
Processing 819b1113ed41ae4cdeaf39c0d8d04a5f200af7c6.ipynb
Processing 2b819c36f10ed302770b358200bde72d21f18f7e.ipynb
Processing 62c

Processing 173e7fc3e03f3bb0e17125288c8261dec3cd6f57.ipynb
Processing dbbe027ffbe786b2d7e637de28ad5e7935aa2dca.ipynb
Processing 2f6d9e8c8dc9fb351ce836a3fbebda628574731b.ipynb
Processing 4ba67a080ec7564d75d0ea1e98666c89e844e57c.ipynb
Processing 67f5df93152ed7a40553c960630682206ffda08a.ipynb
Processing 5cec45378e5b714f32520afd083733e8be3c11e7.ipynb
Processing bbc6f5b43360235f6d897024d0c3e81b627d5dda.ipynb
Processing 769af2055714d391f8783f8be8bf157ca1a62ea3.ipynb
Processing ebe4208c3fdfa855a1cdad9cdd002b2fe1b49b18.ipynb
Processing 08a508580f776bc806ba0b2e4fe97f761b6d8d51.ipynb
Processing 6c3300d69004f27d8f4d79705150f39346d38e7e.ipynb
Processing e8cfebedf8a738e995a7ab07420bcf2e783d9e46.ipynb
Processing 7511e8afb189ff56e5c265152ac728516d7cbc45.ipynb
Processing e50d95b4f0906494a97794ad48b18cdaf3f21cdd.ipynb
Processing 7119dbdc072e3d0ff54bc1fa87e7e4443f08a0e0.ipynb
Processing cc78433e716ce1bd4174cee520f8d73fda721ce0.ipynb
Processing 3675784fc1a7069b0adb80163d4ec6b2ee37826f.ipynb
Processing ae3

Processing 7cb017068ad59dc0a7056be852c5e64ac6a0a5e9.ipynb
Processing 432ab83130f36743d69712d0a7aed4e61f8826fe.ipynb
Processing 42a50a701b8c5cf32bea74cd6c3ef27893b6d58a.ipynb
Processing e2a84368f4ccedb605fae9286cd1b5c0dbeb1877.ipynb
Processing 2da6c795839e9be5ffb59339ecc73183357d8157.ipynb
Processing 6dff33252a0d0d8370f21484db9fcf53fdcfc5e7.ipynb
Processing 0820b0181a2a6872bb8545c7ad8deb4963511890.ipynb
Processing c8a5cd71ac656b3ec121337213bc210d3e60cfee.ipynb
Processing ab54b1e033fbebffad5ef16c9da432019b4931ab.ipynb
Processing 62b0a093ab04818911abacf62cc4f19cae7793d4.ipynb
Processing e854c12203914d074e8c19d940f299feac998cde.ipynb
Processing a8d81e989323db17a49af75ef6e8bfb3a9de942e.ipynb
Processing 024e9b13a2118e754668200e286ab8df20b2b446.ipynb
Processing eb1a6c209bd87a5d242c4cc8fa8e54d9a6c89b44.ipynb
Processing 6bcbac0a7119a23def189646514deb23c79dabfc.ipynb
Processing b6bc412064e9bd2b40d91bab4a42b97c05b58b7a.ipynb
Processing 44a3360c42253ac665518c4fab64d399f19c845f.ipynb
Processing 4d5

Processing 6a4179e136efb042f1605e0cafc59096c7830896.ipynb
Processing 59d0d5a22b308764f45b1d000051506f1219f67e.ipynb
Processing d9e1198667f40183a0dbea121afd875c3b52690c.ipynb
Processing db64d9fbd6b39c88495ad42b15ab2a18f1043ea9.ipynb
Processing ab7332d0f546d49ca9eff91ccfd78168dd865165.ipynb
Processing 3480c916bf0f71f245cb0b4f16c9d6cbbd4378ab.ipynb
Processing 5bafe7213354813290838a56ea7f18eeee90417d.ipynb
Processing a5d762d24681ab9db656afb081d0614e314f0731.ipynb
Processing df8fd113d6787a4a5e5457c13b4f4268d42f77da.ipynb
Processing 39dc3a9171322b7917693bb8f7b6af409635c13b.ipynb
Processing f9dfa51b250bbad5a94e8fcd5798fee02b56f60d.ipynb
Processing 6e011896797a6bb9424eaa122d273b1fe92ebde7.ipynb
Processing 0194311eb0a9cb0b072bf840b44b08259a6dc3ec.ipynb
Processing 3436533a5f676beec65b6c64f1a561995224a1b7.ipynb
Processing 57168c044d65d557bb61555831f0f7df5acab4b9.ipynb
Processing f31be345fba9fa653d307aae9756355c1a44a7ae.ipynb
Processing a35a93a6dd4c7472adb9ae25665682f5addbbee9.ipynb
Processing d04

...

Processing ce81348dc67d1735e17e5e16528cd9568f66a809.ipynb
Processing 9eb046956483f28568f221dd63379752caac619f.ipynb
Processing 48a95a9dd37be175103d6620e910eb731f130a87.ipynb
Processing 6216037915b1cffce70c8aa34dad3146c934270a.ipynb
Processing fe0ec5f40951bd006ada1eba55942136059bcce2.ipynb
Processing d1f62874ac40cec5afa6290e35686f371f9eb09f.ipynb
Processing 183eca912eb674be88e9cbc2b502af647c36a7c9.ipynb
Processing b3d9beba0c4baa537618af8173811d3638d129bb.ipynb
Processing ca26133db354042204d74b036c8f1d5a80257967.ipynb
Processing a0a6a98ba3e5b3c16c12b6bc88ad6f63c8c6c27c.ipynb
Processing fa6619f26ca7e53788074a69c9a3725eef7b27ef.ipynb
Processing f554ebd03a4fd2c4636e4ec17ef8096c680e6ea5.ipynb
Processing 781d68eb3dfb1cf0558de09c49fc35638980f0e1.ipynb
Processing 4b2a3286005907993100243ec01db114c19d76c2.ipynb
Processing 5ad0644c37d785a1d9952d402a68ee8e2d633a0b.ipynb
Processing d71cf2d85e173b9e375cb38a1661c047c9d91551.ipynb
Processing 602a0205f4367f00ec385525fefff77d9d81345d.ipynb
Processing c85

Processing 4d955d95385cba2511c4a20fbbb7bc08f29f6b3e.ipynb
Processing d754ed6af9f0e7fe257fe68def0eb678c27565f7.ipynb
Processing 8f1c619c5ebef1122cdb6908de2e128d55d187c7.ipynb
Processing e832498871afc461eae247f1e8ee8ac24a104024.ipynb
Processing b3577aa688cde100fc689257a941138416a581a2.ipynb
Processing 55d6d15f01520c93e17287ef92a8ebbcbb92e7bd.ipynb
Processing 77c49dbcb96065aa3e844f0d91089d0936a81a00.ipynb
Processing fedbcef33fa14023baa46f3970015f6cd2259027.ipynb
Processing acf38fb6961766f42c3a494204c205b5a07df564.ipynb
Processing be8d0ddf6c85edb7966d0ffc0a0ba30c299446c4.ipynb
Processing f1c0634ec68d562062a06c957f89fbc50edf2848.ipynb
Processing 25abc52d874839a7cca459987c0d0d4533017fb6.ipynb
Processing 02c6d007ace41a973f7a65c3c26b71e5080aae33.ipynb
Processing 5c82d33df923c34cebf1efe9dcefe26dbc2bbe09.ipynb
Processing 9baa83e5b3365705de68760733601ead3e0898dd.ipynb
Processing 242d15fc2ae58cfbefc803c44016c3e679617824.ipynb
Processing 0e3534269fa0c08cfae547d9fee96a31b2151f3f.ipynb
Processing 4f0

Processing 5070ba5f9159bf08b8aade0ffe44b0ac746a56a5.ipynb
Processing d2639396d3d31e6faa3b230964e83d135a28b27a.ipynb
Processing c1f56ed1c6143c2ce21079fb8ae3fa45d63ef451.ipynb
Processing 8f049a2fdd52a08af7b482120698e685c6d2a30d.ipynb
Processing 244c6a749030dbca78d74adb6f6f12a4934963e2.ipynb
Processing 2e96a67a3d5087196fb0055eb22afc3ef31c4c8c.ipynb
Processing f8df13945553f77ca25c6a3d65a053d08ff6ae0a.ipynb
Processing a5a967588ab6668488f18f558f4ef88f83ebf428.ipynb
Processing 1662e5c3b3af813f339ac1810c41fd36e8b41b71.ipynb
Processing 97233cd58ca3f1c9f90f362ef5a5fbfc0884183b.ipynb
Processing 288a23d696593b36495b730b98a024914807702e.ipynb
Processing 5d674aec699e21be692ebbdfbfb054ff4977fbb7.ipynb
Processing c2eeea4aebd8db62831568fb9c2c4578d5918d3f.ipynb
Processing b581a6c1ed228b7957079f6e022bd37702f79a2d.ipynb
Processing bacc45e7dba341856bf64502e10e23ea73752f66.ipynb
Processing 51d5bcf47663a8f8b110630f7185a4b7e8d507d0.ipynb
Processing 19f0a1040e4f04dc1892ff4d861aab8c53726881.ipynb
Processing fbf

Processing 05f2718713bda52b1f82b20566d048b3a47ba6a1.ipynb
Processing 5324425491190e9718ad1bb362fab43a443762f6.ipynb
Processing 61de474ba8ac2b3cbe5f1f9ad6bdd0a92630578a.ipynb
Processing d96f800daca94e972a1a374f81aa783e573f6b76.ipynb
Processing 41d3050b67f3be06f84c8959f5d6ed2964788fe9.ipynb
Processing 10870c0d69a413ab70d033f057fa4a6bb5d5e8eb.ipynb
Processing 24422444c677ce9b23f8bddee4c97bb835904ac5.ipynb
Processing 7bacf2f3efa7c67055278c7fadc21fb7c891b360.ipynb
Processing 5a476a0b6edaade70ceafa4fe71ae0892a039d18.ipynb
Processing 1b6faada833948154cd1ed960eb8b9f243e7c242.ipynb
Processing 83ddad17d736c5d48f3613f5d0649e9d1cef53f1.ipynb
Processing 2b0fe2f1b8fbe1dac34ade31819267f842352f2e.ipynb
Processing 895f3d9cbb10e073a3822257a27e802ff01660b7.ipynb
Processing d2ff2b09f348017d7243078a8f0537c94a0ba630.ipynb
Processing 1b12fc9b3b38ff658f3934f6e37d7b824de182a5.ipynb
Processing 0e7bd38894f0abe2d55b3840181d37ad5d0aa87e.ipynb
Processing 0642ffe85bb43bb96a5a848a623d21332e488f90.ipynb
Processing f0d

/tmp/ipykernel_1618263/307147404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validFiles['imports'] = validFiles.apply(lambda nb: getSourceCodeAndExtractImports(nb), axis=1)


 # Cleanup and save.

In [ ]:
validFiles = validFiles.drop(['raw'], axis=1)
print(validFiles)

validFiles.to_csv('nb_subplots.csv', header=True, index=False)

# write text into a text file
# with open('nbText.txt', 'w') as f:
#     f.write(text)

# validFilesRdd = spark.createDataFrame(validFiles).rdd
# logger.info(validFilesRdd.show())